In [ ]:
# @title Шаг 1: Библиотеки и функции

import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import colorsys
import webcolors



def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))


def rgb_to_cmyk(r, g, b):
    #  RGB в [0, 1]
    r, g, b = r / 255.0, g / 255.0, b / 255.0

    # случай череного цвета
    if (r, g, b) == (0, 0, 0):
        return 0, 0, 0, 100


    k = 1 - max(r, g, b)


    c = (1 - r - k) / (1 - k)
    m = (1 - g - k) / (1 - k)
    y = (1 - b - k) / (1 - k)

    return c * 100, m * 100, y * 100, k * 100

def cmyk_to_rgb(c, m, y, k):
    #  CMYK в [0, 1]
    c, m, y, k = c / 100.0, m / 100.0, y / 100.0, k / 100.0

    r = 255 * (1 - c) * (1 - k)
    g = 255 * (1 - m) * (1 - k)
    b = 255 * (1 - y) * (1 - k)
    return r, g, b


def rgb_to_hsv(r, g, b):
    h, s, v = colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)
    return h * 360, s * 100, v * 100

def hsv_to_rgb(h, s, v):
    r, g, b = colorsys.hsv_to_rgb(h / 360.0, s / 100.0, v / 100.0)
    return r * 255, g * 255, b * 255


def rgb_to_hls(r, g, b):
    h, l, s = colorsys.rgb_to_hls(r / 255.0, g / 255.0, b / 255.0)
    return h * 360, l * 100, s * 100

def hls_to_rgb(h, l, s):
    r, g, b = colorsys.hls_to_rgb(h / 360.0, l / 100.0, s / 100.0)
    return r * 255, g * 255, b * 255


In [10]:
# @title Запуск приложения для нечетных вариантов (CMYK-RGB-HLS)

_UPDATING = False

# Работа над отображением
def create_control_group(label, min_val, max_val, is_float=False):
    slider_class = widgets.FloatSlider if is_float else widgets.IntSlider
    text_class = widgets.FloatText if is_float else widgets.IntText
    text_kwargs = {'format': '.3f'} if is_float else {}

    label_widget = widgets.Label(label, layout=widgets.Layout(width='30px'))
    slider_widget = slider_class(min=min_val, max=max_val, step=0.1 if is_float else 1, layout=widgets.Layout(width='200px'))
    text_widget = text_class(min=min_val, max=max_val, layout=widgets.Layout(width='70px'), **text_kwargs)

    widgets.jslink((slider_widget, 'value'), (text_widget, 'value'))

    return widgets.HBox([label_widget, slider_widget, text_widget]), slider_widget, text_widget


# --- Создание UX/UI ---
color_picker = widgets.ColorPicker(concise=False, description='Выбрать цвет', value='#4169E1')
color_display_output = widgets.Output(layout=widgets.Layout(width='150px', height='150px', border='1px solid black'))

with color_display_output:
    fig, ax = plt.subplots(figsize=(2, 2))
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout(pad=0)
    plt.show()

rgb_title = widgets.HTML("<h3>RGB</h3>")
r_box, r_slider, r_text = create_control_group('R:', 0, 255)
g_box, g_slider, g_text = create_control_group('G:', 0, 255)
b_box, b_slider, b_text = create_control_group('B:', 0, 255)
rgb_controls = widgets.VBox([rgb_title, r_box, g_box, b_box])


cmyk_title = widgets.HTML("<h3>CMYK</h3>")
c_box, c_slider, c_text = create_control_group('C:', 0, 100, is_float=True)
m_box, m_slider, m_text = create_control_group('M:', 0, 100, is_float=True)
y_box, y_slider, y_text = create_control_group('Y:', 0, 100, is_float=True)
k_box, k_slider, k_text = create_control_group('K:', 0, 100, is_float=True)
cmyk_controls = widgets.VBox([cmyk_title, c_box, m_box, y_box, k_box])


hls_title = widgets.HTML("<h3>HLS</h3>")
h_box_hls, h_slider_hls, h_text_hls = create_control_group('H:', 0, 360, is_float=True)
l_box_hls, l_slider_hls, l_text_hls = create_control_group('L:', 0, 100, is_float=True)
s_box_hls, s_slider_hls, s_text_hls = create_control_group('S:', 0, 100, is_float=True)
hls_controls = widgets.VBox([hls_title, h_box_hls, l_box_hls, s_box_hls])


def update_all_colors(change):
    global _UPDATING
    if _UPDATING:
        return
    _UPDATING = True

    try:
        owner = change['owner']
        r, g, b = 0, 0, 0

        if owner in [r_slider, g_slider, b_slider]:
            r, g, b = r_slider.value, g_slider.value, b_slider.value
        elif owner in [c_slider, m_slider, y_slider, k_slider]:
            r, g, b = cmyk_to_rgb(c_slider.value, m_slider.value, y_slider.value, k_slider.value)
        elif owner in [h_slider_hls, l_slider_hls, s_slider_hls]:
            r, g, b = hls_to_rgb(h_slider_hls.value, l_slider_hls.value, s_slider_hls.value)
        elif owner == color_picker:
            r, g, b = hex_to_rgb(owner.value)
        else:
            _UPDATING = False
            return

        c, m, y, k = rgb_to_cmyk(r, g, b)
        h, l, s = rgb_to_hls(r, g, b)
        hex_val = rgb_to_hex((r, g, b))

        # Обновление виджетов
        r_slider.value, g_slider.value, b_slider.value = r, g, b
        c_slider.value, m_slider.value, y_slider.value, k_slider.value = c, m, y, k
        h_slider_hls.value, l_slider_hls.value, s_slider_hls.value = h, l, s
        color_picker.value = hex_val

        # Обновляем квадратик
        with color_display_output:
            color_display_output.clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(2, 2))
            ax.set_facecolor(hex_val)
            ax.set_xticks([])
            ax.set_yticks([])
            plt.tight_layout(pad=0)
            plt.show()

    finally:
        _UPDATING = False


widgets_to_observe = [
    r_slider, g_slider, b_slider,
    c_slider, m_slider, y_slider, k_slider,
    h_slider_hls, l_slider_hls, s_slider_hls,
    color_picker
]

for widget in widgets_to_observe:
    widget.observe(update_all_colors, names='value')


all_sliders = widgets.VBox([rgb_controls, cmyk_controls, hls_controls])
left_panel = widgets.VBox([color_picker, color_display_output])
app_layout = widgets.HBox([left_panel, all_sliders])


initial_r, initial_g, initial_b = hex_to_rgb(color_picker.value)
update_all_colors({'owner': color_picker, 'new': color_picker.value})

display(HTML("<h2>Интерактивный конвертер цветов (CMYK-RGB-HLS)</h2>"))
display(app_layout)

